## Implementing Multi-Head Attention with Weight Splits

In [3]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [4]:
torch.manual_seed(123)

inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66], 
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33], 
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
context_vecs

torch.Size([2, 3, 6])


tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>)

### Step by Step execution of MHA

In [9]:
x = torch.tensor(
    [[[1.0, 2.0, 3.0, 4.0, 5.0, 6.0], 
      [6.0, 5.0, 4.0, 3.0, 2.0, 1.0], 
      [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]]
)
x.shape # Shape of x is (batch_size, num_tokens, d_in)

torch.Size([1, 3, 6])

In [15]:
b, num_tokens, d_in = x.shape
d_out = d_in
num_heads = 2
head_dim = d_out//num_heads
context_length = num_tokens

In [10]:
W_query = nn.Linear(d_in, d_out, bias=False)
W_key = nn.Linear(d_in, d_out, bias=False)
W_value = nn.Linear(d_in, d_out, bias=False)
W_key.weight

Parameter containing:
tensor([[-0.2350,  0.2106,  0.2885, -0.3961, -0.3465, -0.3976],
        [ 0.1540,  0.3286, -0.3165, -0.1890,  0.1299, -0.2666],
        [ 0.3468,  0.0952, -0.1136,  0.0265,  0.1273, -0.1444],
        [-0.3163,  0.0027,  0.0074,  0.0082, -0.0596,  0.2621],
        [-0.1139, -0.0395,  0.1679, -0.2570,  0.1093, -0.0903],
        [ 0.1958, -0.2215,  0.0151,  0.0399, -0.3285, -0.2969]],
       requires_grad=True)

In [11]:
queries = W_query(x)
keys = W_key(x)
values = W_value(x)

In [12]:
# We implicitly split the matrix by adding a `num_heads` dimension
# Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
keys = keys.view(b, num_tokens, num_heads, head_dim) 
values = values.view(b, num_tokens, num_heads, head_dim)
queries = queries.view(b, num_tokens, num_heads, head_dim)

In [13]:
# Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
keys = keys.transpose(1, 2)
queries = queries.transpose(1, 2)
values = values.transpose(1, 2)

In [14]:
# Compute scaled dot-product attention (aka self-attention) with a causal mask
attn_scores = queries @ keys.transpose(2, 3)

In [16]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# Original mask truncated to the number of tokens and converted to boolean
mask_bool = mask.bool()[:num_tokens, :num_tokens]

In [19]:
attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head
attn_scores.masked_fill_(mask_bool, -torch.inf)
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

In [20]:
# Shape: (b, num_tokens, num_heads, head_dim)
context_vec = (attn_weights @ values).transpose(1, 2)

In [22]:
# Combine heads, where self.d_out = self.num_heads * self.head_dim
# Now the shape of context_vec is same as the shape of inputs
context_vec = context_vec.contiguous().view(b, num_tokens, d_out)